In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [2]:
df_transactions = pd.read_csv(x
                                    "Thalia_transactions_ersteBereinigung.csv")

In [3]:
df_privatkunden_iids = pd.read_csv(x+
                                    "privatkunden.iid.thalia.csv", index_col = [0])

##### Privatkunden und Trans mergen

In [4]:
# Privatkunden iids mit Transaktionsdaten mergen
df_t = pd.merge(df_transactions, df_privatkunden_iids, on = ["iid"])
df_t.head()

,iid,date,month_count
0,100002747,2022-02-03 12:44:22,21
1,100002747,2022-04-29 14:03:00,23
2,100009615,2018-09-10 09:52:00,-20
3,100009615,2018-10-22 10:32:00,-19
4,100011333,2022-05-27 10:24:33,24


##### Auswahl der Kunden: Transaktion im Oktober 2021 und mindestes eine Transaktion nach Mai 2022

In [7]:
# Alle "iid" mit "month_count" = 0
iid_month_0 = df_t.groupby("iid").filter(lambda x: (x["month_count"] == 17).any())["iid"].unique()

# Alle "iid" mit "month_count" >= 7
iid_month_7 = df_t.groupby("iid").filter(lambda x: (x["month_count"] >= 23).any())["iid"].unique()

# Die Schnittmenge beider Listen gibt alle "iid" aus, die beide Bedingungen erfüllen
iid_both = list(set(iid_month_0) & set(iid_month_7))

##### Mergen der Auswahl mit Trans_df

In [8]:
# Annahme: iid_both ist eine Liste von eindeutigen iids
df_t_filtered = df_t[df_t['iid'].isin(iid_both)]

##### Filter nach Trans nach t > 16

In [9]:
# Ausschließlich Transaktionen nach t > -1 relevant
df_t_filtered = df_t_filtered[df_t_filtered["month_count"] > 16]

In [10]:
minimum_date = df_t_filtered['date'].min()
print(minimum_date)

2021-10-01 08:22:00


#### Speichern der Daten

In [14]:
path = x
filename = 'Thalia_transactions_zweiteBereinigung.csv'

df_t_filtered.to_csv(os.path.join(path, filename), index=False)

#### Ermittlung relevanter iids

In [15]:
path = x
filename = 'Thalia_transactions_zweiteBereinigung_iids.csv'

df_iid_both = pd.DataFrame(iid_both, columns=["iid"])
df_iid_both.to_csv(os.path.join(path, filename), index=False)